## Tutorial on how to use `timestaps` in Field construction

In [1]:
from parcels import Field
from glob import glob
import numpy as np

Some NetCDF files, such as for example those from the [World Ocean Atlas](https://www.nodc.noaa.gov/OC5/woa18/), have time calendars that can't be parsed by `xarray`. These result in a `ValueError: unable to decode time units`, for example when the calendar is in 'months since' a particular date.

In these cases, a workaround in Parcels is to use the `timestamps` argument in `Field` (or `FieldSet`) creation. Here, we show how this works for example temperature data from the World Ocean Atlas in the Pacific Ocean

The following cell raises an error, since the calendar of the World Ocean Atlas data is in "months since 1955-01-01 00:00:00"

In [2]:
tempfield = Field.from_netcdf(glob('WOA_data/woa18_decav_*_04.nc'), 't_an', 
                              {'lon': 'lon', 'lat': 'lat', 'time': 'time'})

         It will be opened with no decoding. Filling values might be wrongly parsed.


RuntimeError: Xarray could not convert the calendar. If youre using from_netcdf, try using the timestamps keyword in the construction of your Field. See also the tutorial at https://nbviewer.jupyter.org/github/OceanParcels/parcels/blob/master/parcels/examples/tutorial_timestamps.ipynb

However, we can create our own numpy array of timestamps associated with each of the 12 snapshots in the netcdf file

In [3]:
timestamps = np.expand_dims(np.array([np.datetime64('2001-%.2d-15' %m) for m in range(1,13)]), axis=1)

And then we can add the `timestamps` as an extra argument

In [4]:
tempfield = Field.from_netcdf(glob('WOA_data/woa18_decav_*_04.nc'), 't_an',
                              {'lon': 'lon', 'lat': 'lat', 'time': 'time'},
                              netcdf_decodewarning=False,
                              timestamps=timestamps)

Note, by the way, that adding the `time_periodic=True` argument to `Field.from_netcdf()` will also mean that the climatology can be cycled for multiple years.

Furthermore, note that we used `netcdf_decodewarning=False` in the `FieldSet.from_netcdf()` call above. This is to silence an expected warning because the time dimension in the `coordinates.nc` file can't be decoded by `xarray`.